# Capstone Project - The Battle of the Neighborhoods - task 2 (Week 5)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to explore and find the pattern for each borough and neighborhood in Amsterdam, which area is famous for something and how similar each neighborhood or borough. This information maybe useful for tourist companies, or even new start-up if they would like to start doing business in Netherlands, specifically here is Amsterdam.

Since there are 8 boroughs in the Amsterdam, first we need to sampling them by try to organizing the data obtain from the Internet and fix some wrong data. It is really difficult to say which Borough and neighborhoods have similar patterns like: restaurants, museum, park area, bus stop, ....

The data science power will be used in order to exploit the advantage of each area, and show the similar of those areas, then this would help to reduce time when planning to travel Amsterdam in smart way, or try to find out which area is best suit for invest some start-up company.

## Data <a name="data"></a>

Based on the previous section discussion, there are some factors will effect our decision such as:
* Number of venues that we can obtains
* The diverse ecology venues that Amsterdam having
* Number of neighborhoods, and how dense the population are 

First the data will be obtain by extract information how Amsterdam, Netherlands are formed administratively and separated into different small regions, this can be like sampling the Amsterdam areas equally in such a way we can easily to explore using some external API that is very useful.

Number of venues with certain radius will be obtain by using Foursquare API.
coordinate of Amsterdam center will be obtained using Google Maps API geocoding
Some wrong locations will be fixed by manually adding using google maps.

### Neighborhoods acquisition 
* to obtain data, i use the link on wiki containing boroughs and neiborhoods of Amsterdams 'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam#List_of_boroughs' , and then, scrape the table with meaningful information inside.
* After the borough and neighborhoods are found, we use 'geopy' library with method like 'geocode' to translate name place into latitude and longitude coordination with can be very useful later to interact with Square API to extract all the nearby Venues in the place we want to find out. 
* And following step can be found in the link of another jupiter notebook at: https://github.com/MossyFighting/Coursera_Capstone/blob/master/raw_data.ipynb
* However, one notice is that, in the data neighborhoods, There are some fields the 'geopy' could not find the coordinations for latitude and longitude, so then i have to fill it by manually search each one. Howerver, that are not so many (among them six are wrong coordination). And the final data will be attached with name 'Amsterdam.csv'!
* After collecting all locations data coordination, we will used folium library to plot those locations into real maps (popular/famous maps look like google maps, or Bing, ...) to check whether our collections are actually true or not.
* Indeed, looking into the plot after generating the location coordination, we can reconfirm that our collection is true. because all the blue circle lie within the Amsterdam city area. 

In [3]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import numpy as np
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

# read data
df = pd.read_csv('Amsterdam.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.rename(columns = {'0' : 'Latitude', '1' : 'Longitude', 'Neighborhoods' : 'Neighborhood' }, inplace = True)
print(df.head())

# plot map amsterdam with sampling points are the neighborhoods 
# latitude and longitude of Amsterdam to create map 
latitude = 52.3545362
longitude = 4.7638781

# create map of Amsterdam using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
map_amsterdam

Libraries imported.
  Borough           Neighborhood   Latitude  Longitude
0    West  Frederik Hendrikbuurt  52.376956   4.874085
1    West              Houthaven  52.393376   4.881680
2    West      Spaarndammerbuurt  52.389663   4.879369
3    West      Staatsliedenbuurt  52.380286   4.870951
4    West         Zeeheldenbuurt  52.389330   4.888242


### Foursquare API
The data location coordinations are available for us to use. But the questions are now is, how to use that data locations, and which tools should we use to extract some meaningful informations such as: how many restaurants in each neighborhood or borough, and so on. 

The answers to those questions will be the Foursquare API (that i decided to choose for this project) which is a strong tool and very impressive to returns all the possible venues that are near the neighborhoods that we collected in previous sections.

Because we would like to find the similar or dissimilar patterns between those neighborhoods, then we will keep all the venues, we will not filter out any categories that return from our call. 

the first thing to remember when using fourth square api is to define apps informations such as: user identification and user credentials that is registered in the main Square website. 

In [4]:
# Define foursquare API credential to use API
CLIENT_ID = 'MC2SRBWTQ4OJXCCA1KUMROH2SOVUVP103QMRIRDMPJHEPGNK' # Foursquare ID
CLIENT_SECRET = 'IFMJISV3YMKFQI1WTGAUAKTKPQ3EUPBOO4J2BQ55Z1M1WRQY' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# function to return nearby venues for any latitude and longitude
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create data for each Borough
center_data = df[df['Borough'] == 'Centrum (Centre)'].reset_index(drop=True)
west_data = df[df['Borough'] == 'West'].reset_index(drop=True)
south_data = df[df['Borough'] == 'Zuid (South)'].reset_index(drop=True)
east_data = df[df['Borough'] == 'Oost (East)'].reset_index(drop=True)
newwest_data = df[df['Borough'] == 'Nieuw-West(New West)'].reset_index(drop=True)
southeast_data = df[df['Borough'] == 'Zuidoost(Southeast)'].reset_index(drop=True)
North_data = df[df['Borough'] == 'Noord (North)'].reset_index(drop=True)
westgateway_data = df[df['Borough'] == 'Westpoort(West Gateway)'].reset_index(drop=True)

# set limit venues and radius to explore
LIMIT = 150
radius = 600

Your credentails:
CLIENT_ID: MC2SRBWTQ4OJXCCA1KUMROH2SOVUVP103QMRIRDMPJHEPGNK
CLIENT_SECRET:IFMJISV3YMKFQI1WTGAUAKTKPQ3EUPBOO4J2BQ55Z1M1WRQY


### Acquisition all venues from neighborhoods
* Looking from the data table we can see that the center having the most density and small area with lots of canals. Then we would like to explore this area.

In [11]:
# neighborhoods in center area
print('neighborhoods in center area')
center_venues = getNearbyVenues(names=center_data['Neighborhood'],
                                   latitudes=center_data['Latitude'],
                                   longitudes=center_data['Longitude']
                                  )
print('print out the sampling points of venues at center area:', center_venues.head())

# check venues data frame and compute number of unique features 
print('* size of the resulting dataframe', center_venues.shape)
print(center_venues.head())
print('* venues were returned for each neighborhood', center_venues.groupby('Neighborhood').count())
print('* There are {} uniques categories.'.format(len(center_venues['Venue Category'].unique())))


# one hot encoder for venue data frames
# one hot encoding
center_onehot = pd.get_dummies(center_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
center_onehot['Neighborhood'] = center_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [center_onehot.columns[-1]] + list(center_onehot.columns[:-1])
center_onehot = center_onehot[fixed_columns]

#print(center_onehot.head())
print(center_onehot.shape)

# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
center_grouped = center_onehot.groupby('Neighborhood').mean().reset_index()
print('mean of frequency of occurence of each group:', center_grouped.head())


neighborhoods in center area
Binnenstad
Grachtengordel
Haarlemmerbuurt
Jodenbuurt
Jordaan
Kadijken
Lastage
Oosterdokseiland
Oostelijke Eilanden
Plantage
Rapenburg
Uilenburg
Westelijke Eilanden
Weteringschans
print out the sampling points of venues at center area:   Neighborhood  Neighborhood Latitude  Neighborhood Longitude          Venue  \
0   Binnenstad              52.373128                 4.88808  De Bierkoning   
1   Binnenstad              52.373128                 4.88808     The Hoxton   
2   Binnenstad              52.373128                 4.88808    The Duchess   
3   Binnenstad              52.373128                 4.88808    W Amsterdam   
4   Binnenstad              52.373128                 4.88808    Hummingbird   

   Venue Latitude  Venue Longitude     Venue Category  
0       52.372404         4.889795         Beer Store  
1       52.371863         4.887487              Hotel  
2       52.372712         4.889253  French Restaurant  
3       52.372787         4.890

### Explore the resulting frame

In [13]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_interest_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_interest_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = center_grouped['Neighborhood']

for ind in np.arange(center_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(center_grouped.iloc[ind, :], num_interest_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Binnenstad,Hotel,Café,Bar,French Restaurant,Dessert Shop,Bookstore,Marijuana Dispensary,Thrift / Vintage Store,Steakhouse,Italian Restaurant,Furniture / Home Store,Plaza,Beer Bar,Sandwich Place,Bakery,Boutique,Clothing Store,Cocktail Bar,Hotel Bar,Coffee Shop
1,Grachtengordel,Hotel,Bar,Restaurant,Bookstore,French Restaurant,Cheese Shop,Chocolate Shop,Sandwich Place,Café,Coffee Shop,Bakery,Italian Restaurant,Canal,Pub,Plaza,Creperie,Seafood Restaurant,Clothing Store,Hotel Bar,Cosmetics Shop
2,Haarlemmerbuurt,Bar,Café,Italian Restaurant,Marijuana Dispensary,Deli / Bodega,Restaurant,Yoga Studio,Organic Grocery,Tapas Restaurant,Sandwich Place,Supermarket,Steakhouse,Plaza,Comfort Food Restaurant,Coffee Shop,Optical Shop,Cheese Shop,Canal,Pub,Ramen Restaurant
3,Jodenbuurt,Marijuana Dispensary,Hotel,Bar,Italian Restaurant,Coffee Shop,Café,Theater,Sandwich Place,Gastropub,Brewery,Plaza,History Museum,Steakhouse,Bagel Shop,Movie Theater,Cocktail Bar,Multiplex,Opera House,Chinese Restaurant,Outdoor Sculpture
4,Jordaan,Bar,Café,Coffee Shop,Hotel,Furniture / Home Store,Pizza Place,Thai Restaurant,Museum,Record Shop,Restaurant,Salad Place,Italian Restaurant,Indonesian Restaurant,Boutique,Indian Restaurant,Bistro,Yoga Studio,Gym / Fitness Center,Sandwich Place,French Restaurant


## Methodology <a name="methodology"></a>
* In this assignment, we will try to find the patterns in each borough, actually, will only explore and find out the most famous area, we will not cover all borough areas. 
* Next step, we will see what kind of venues in this area that we are interests. What is the most interest, the most common venue. and how similar between neighborhoods.
* Finally, we will use K-means clustering in machine learning algorithms to build a model, and apply to out data set obtain with those features from categories in all neighborhoods to see how similar they are, and how different they are.
* In order to have a clear compare between boroughs, i will try to analyze another borough with high density, and see how they different with the center area.

## Analysis <a name="analysis"></a>
We will extract the most common venue in each neighborhood.

In [15]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = center_grouped['Neighborhood']

for ind in np.arange(center_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(center_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Binnenstad,Hotel,Café,Bar,French Restaurant,Dessert Shop,Bookstore,Marijuana Dispensary,Thrift / Vintage Store,Steakhouse,Italian Restaurant
1,Grachtengordel,Hotel,Bar,Restaurant,Bookstore,French Restaurant,Cheese Shop,Chocolate Shop,Sandwich Place,Café,Coffee Shop
2,Haarlemmerbuurt,Bar,Café,Italian Restaurant,Marijuana Dispensary,Deli / Bodega,Restaurant,Yoga Studio,Organic Grocery,Tapas Restaurant,Sandwich Place
3,Jodenbuurt,Marijuana Dispensary,Hotel,Bar,Italian Restaurant,Coffee Shop,Café,Theater,Sandwich Place,Gastropub,Brewery
4,Jordaan,Bar,Café,Coffee Shop,Hotel,Furniture / Home Store,Pizza Place,Thai Restaurant,Museum,Record Shop,Restaurant
5,Kadijken,Zoo Exhibit,Restaurant,Plaza,Supermarket,Italian Restaurant,Hotel,Science Museum,Planetarium,Park,Museum
6,Lastage,Bar,Hotel,Chinese Restaurant,Hostel,Italian Restaurant,Marijuana Dispensary,Cocktail Bar,Bagel Shop,Café,Coffee Shop
7,Oostelijke Eilanden,Park,Gym / Fitness Center,Café,Bus Stop,Breakfast Spot,Mediterranean Restaurant,Pub,Hotel,Restaurant,Seafood Restaurant
8,Oosterdokseiland,Hotel,Restaurant,Bar,Hotel Bar,Italian Restaurant,Hostel,Pub,Boat or Ferry,Steakhouse,Coffee Shop
9,Plantage,Zoo Exhibit,Bar,Hotel,Café,Pizza Place,Breakfast Spot,Restaurant,Museum,History Museum,Italian Restaurant


### segmentation to understanding about pattern
* among 13 neighborhoods, now we will try to segment them into different group to see how different/similar they are.
* Because the number of observations that are the number of rows in our data is too small, so we are not able to split data into train or test, in another way, i just randomly choose the number of clusters based on test and trial with the most accuracy.
* then the clusters after segmentation will be plot in to map for easy visualization. 

In [16]:
# segmentation
# set number of clusters
kclusters = 3

center_grouped_clustering = center_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(center_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print('labels after running clustering', kmeans.labels_[0:10]) 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

center_merged = center_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
center_merged = center_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(center_merged.head()) # check the last columns!

# longitude and latitude of center of amsterdam
latitude = 52.3733713
longitude = 4.8689007

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(center_merged['Latitude'], center_merged['Longitude'], center_merged['Neighborhood'], center_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

labels after running clustering [1 1 1 1 1 2 1 0 1 1]
            Borough     Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Centrum (Centre)       Binnenstad  52.373128   4.888080               1   
1  Centrum (Centre)   Grachtengordel  52.370837   4.885478               1   
2  Centrum (Centre)  Haarlemmerbuurt  52.382441   4.887193               1   
3  Centrum (Centre)       Jodenbuurt  52.368870   4.900223               1   
4  Centrum (Centre)          Jordaan  52.375416   4.881096               1   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                 Hotel                  Café                   Bar   
1                 Hotel                   Bar            Restaurant   
2                   Bar                  Café    Italian Restaurant   
3  Marijuana Dispensary                 Hotel                   Bar   
4                   Bar                  Café           Coffee Shop   

  4th Most Common Venue   5th Most Common Venue 6t

In [19]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 0, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Oostelijke Eilanden,Park,Gym / Fitness Center,Café,Bus Stop,Breakfast Spot,Mediterranean Restaurant,Pub,Hotel,Restaurant,Seafood Restaurant


In [20]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 1, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Binnenstad,Hotel,Café,Bar,French Restaurant,Dessert Shop,Bookstore,Marijuana Dispensary,Thrift / Vintage Store,Steakhouse,Italian Restaurant
1,Grachtengordel,Hotel,Bar,Restaurant,Bookstore,French Restaurant,Cheese Shop,Chocolate Shop,Sandwich Place,Café,Coffee Shop
2,Haarlemmerbuurt,Bar,Café,Italian Restaurant,Marijuana Dispensary,Deli / Bodega,Restaurant,Yoga Studio,Organic Grocery,Tapas Restaurant,Sandwich Place
3,Jodenbuurt,Marijuana Dispensary,Hotel,Bar,Italian Restaurant,Coffee Shop,Café,Theater,Sandwich Place,Gastropub,Brewery
4,Jordaan,Bar,Café,Coffee Shop,Hotel,Furniture / Home Store,Pizza Place,Thai Restaurant,Museum,Record Shop,Restaurant
6,Lastage,Bar,Hotel,Chinese Restaurant,Hostel,Italian Restaurant,Marijuana Dispensary,Cocktail Bar,Bagel Shop,Café,Coffee Shop
7,Oosterdokseiland,Hotel,Restaurant,Bar,Hotel Bar,Italian Restaurant,Hostel,Pub,Boat or Ferry,Steakhouse,Coffee Shop
9,Plantage,Zoo Exhibit,Bar,Hotel,Café,Pizza Place,Breakfast Spot,Restaurant,Museum,History Museum,Italian Restaurant
10,Rapenburg,Bar,Hotel,Hostel,Coffee Shop,Italian Restaurant,Pizza Place,Breakfast Spot,History Museum,Boat or Ferry,Chinese Restaurant
11,Uilenburg,Bar,Hotel,Italian Restaurant,Marijuana Dispensary,Hostel,Bagel Shop,Café,Chinese Restaurant,Coffee Shop,Pizza Place


In [22]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 2, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kadijken,Zoo Exhibit,Restaurant,Plaza,Supermarket,Italian Restaurant,Hotel,Science Museum,Planetarium,Park,Museum


## Results and Discussion <a name="results"></a>
* As we can see from above frames, the group 0, is the neighbor named 'Oostelijke Eilanden' is populated with natural and friedly venues such as gyms, parks and bus stop. Whereas, in the group 1, they are mostly populated with food and beverage groups with so many bars, cafe, and Hotels.  
* And in the group 3, we can easu


## Conclusion <a name="conclusion"></a>

The goal of this project is to aim to exploration and see the patterns of Amsterdam so that in the final we can have at least some information about the city where someone would like to explore or visit in the future. To fulfill that, we have retrieve data about boroughs of only Amsterdam area where we are interested in, and collect some information such a longitude and latitude which are mandatory because we would like to use Foursquare API to retrive the data about venues around Amsterdam. 

In addition, we had explore some common places in all neighborhoods of central area where the population per square kilometer is dense and  there are a lot of interesting venue to visit.